<a href="https://colab.research.google.com/github/sjbaek12/sjbaek12.github.io/blob/master/%EA%B0%95%ED%99%94%ED%95%99%EC%8A%B5_%EC%98%88%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt






In [ ]:
import gym
env = gym.make("CartPole-v0")


In [ ]:
# Try running environment with random actions

# CartPole은 막대기를 쓰러뜨리지 않고 옮기는 것인데 매 에피소드마다 최대보상이 200이고, 
# 매번 step마다 1의 보상을 얻는데 100에피소드를 해서 모두 195점 이상 얻으면 학습이 완료된 것으로 인정한다.

env.reset()  # 새로운 에피소드를 불러온다.
reward_sum = 0
num_games = 10
num_game = 0
while num_game < num_games:
#     env.render()  # 행동을 하기전 환경에 대해 얻은 관찰값을 그린다.
    observation, reward, done, _ = env.step(env.action_space.sample())
    # env.action_space.sample() 환경에서 행동 샘플링이다. 1은 오른쪽 이동 , 0은 왼쪽 이동이다.
    # env.step 행동을 취한 후 환경에서 얻은 관찰값으로 제어
    # observation은 4개의 실수로 구성된 배열로 state를 나타낸다
    # done은 episode가 종료되었을때 True 이다. 막대기가 쓰러지면 종료된다.
    print(observation, reward, done)
    reward_sum += reward
    if done:
        print("Reward for this episode was: {}".format(reward_sum))
        reward_sum = 0
        num_game += 1
        env.reset()


[ 0.02733558 -0.19326005  0.04402046  0.35086633] 1.0 False
[0.02347038 0.00120913 0.05103778 0.07238298] 1.0 False
[ 0.02349456  0.19556364  0.05248544 -0.20377058] 1.0 False
[ 0.02740583  0.38989724  0.04841003 -0.47944594] 1.0 False
[ 0.03520378  0.58430355  0.03882111 -0.75648678] 1.0 False
[ 0.04688985  0.77886953  0.02369138 -1.03670532] 1.0 False
[ 0.06246724  0.5834408   0.00295727 -0.73667987] 1.0 False
[ 0.07413606  0.38827813 -0.01177633 -0.44306771] 1.0 False
[ 0.08190162  0.19332478 -0.02063768 -0.15412012] 1.0 False
[ 0.08576811  0.38873605 -0.02372008 -0.45324166] 1.0 False
[ 0.09354284  0.58418526 -0.03278492 -0.75330604] 1.0 False
[ 0.10522654  0.38953031 -0.04785104 -0.47111757] 1.0 False
[ 0.11301715  0.58529434 -0.05727339 -0.77849032] 1.0 False
[ 0.12472303  0.39100471 -0.0728432  -0.50436292] 1.0 False
[ 0.13254313  0.19698088 -0.08293045 -0.23549593] 1.0 False
[ 0.13648275  0.39318374 -0.08764037 -0.55314238] 1.0 False
[ 0.14434642  0.19939476 -0.09870322 -0.2893

In [ ]:
# 이과정은 성과가 좋은 행동을 mimicking하는 것이다.

import keras.layers as layers
from keras.models import Model
from keras.optimizers import Adam
import keras.backend as K  # 케라스의 backend를 K. 형식으로 호출하는 것이고 여기서는 tensorflow이다.
from keras.initializers import glorot_uniform

def get_policy_model(env, hidden_layer_neurons, lr):
    dimen = env.reset().shape # 환경의 shape이고 이것은 4개의 배열로 구성된다.
    num_actions = env.action_space.n  # 가능한 액션의 숫자이다. 여기서는 2로 오른쪽이나 왼쪽으로 움직이는 것
    
    adv = layers.Input(shape=[1], name="advantages")

    # 네개 배열로 구성된 환경이 망으로 들어가서 2개의 가능한 액션을 산출하는 신경망이다.
    inp = layers.Input(shape=dimen,name="input_x")
    x = layers.Dense(hidden_layer_neurons, 
                     activation="relu", 
                     use_bias=False,
                     kernel_initializer=glorot_uniform(seed=42),
                     name="dense_1")(inp)
    out = layers.Dense(num_actions, 
                       activation="softmax", 
                       kernel_initializer=glorot_uniform(seed=42),
                       use_bias=False,
                       name="out")(x)

    def custom_loss(y_true, y_pred):
        # actual: 0 predict: 0 -> log(0 * (0 - 0) + (1 - 0) * (0 + 0)) = -inf
        # actual: 1 predict: 1 -> log(1 * (1 - 1) + (1 - 1) * (1 + 1)) = -inf
        # actual: 1 predict: 0 -> log(1 * (1 - 0) + (1 - 1) * (1 + 0)) = 0
        # actual: 0 predict: 1 -> log(0 * (0 - 1) + (1 - 0) * (0 + 1)) = 0
        log_lik = K.log(y_true * (y_true - y_pred) + (1 - y_true) * (y_true + y_pred))
        return K.mean(log_lik * adv, keepdims=True)
        
    model_train = Model(inputs=[inp, adv], outputs=out)  # state가 input되고 action이 output이다. 
    model_train.compile(loss=custom_loss, optimizer=Adam(lr)) # 그러나 손실함수가 커스텀이다. output이 두개이므로 미분 가능한 형태로 하나로 합쳐서 만들어낸 것이다.
    model_predict = Model(inputs=[inp], outputs=out)
    return model_train, model_predict  # 이 함수의 산출물은 모델 자체와 예측값이다.

In [ ]:
def discount_rewards(r, gamma=0.99):
    """Takes 1d float array of rewards and computes discounted reward
    e.g. f([1, 1, 1], 0.99) -> [2.9701, 1.99, 1]
    """
    prior = 0
    out = []
    for val in r:
        new_val = val + prior * gamma
        out.append(new_val)
        prior = new_val
    return np.array(out[::-1])

In [ ]:
# Constants defining our neural network
hidden_layer_neurons = 8
gamma = .99
dimen = len(env.reset())
print_every = 100
batch_size = 50
num_episodes = 10000
render = False
lr = 1e-2
goal = 100  # 않쓰러지는 목표 점수 같음

In [ ]:
# See our trained bot in action, 테스트 단계이다. 기존에 랜덤하게 만들어진 상황에 최상의 성과를 가져오는 액션을 산출하는 
# 정책신경망에 random하게 만들어진 새로운 환경을 주입하고 그 것이 어떤 성과를 가져오는지 계산하는 것이다.
def score_model(model, num_tests, render=False):
    scores = []    
    for num_test in range(num_tests):
        observation = env.reset()
        reward_sum = 0
        while True:
            if render:
                env.render() # 행동을 하기전 환경에 대해 얻은 관찰값을 그린다

            state = np.reshape(observation, [1, dimen]) # random하게 상태를 다시 만들어 내고
            predict = model.predict([state])[0] # 그 상태에서 행동을 다시 예측한다.
            action = np.argmax(predict) # 예측값중에서 몇번째가 최대값인지 알아낸다. 즉 random하게 한 행동 중에서 어떤 것이 최대의 성과를 냇는지 알아내는 것이다.
            observation, reward, done, _ = env.step(action)
            reward_sum += reward
            if done:
                break
        scores.append(reward_sum)
    env.close()
    return np.mean(scores)

In [ ]:
model_train, model_predict = get_policy_model(env, hidden_layer_neurons, lr)
model_predict.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_x (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 32        
_________________________________________________________________
out (Dense)                  (None, 2)                 16        
Total params: 48
Trainable params: 48
Non-trainable params: 0
_________________________________________________________________


In [ ]:
reward_sum = 0

num_actions = env.action_space.n

# Placeholders for our observations, outputs and rewards
states = np.empty(0).reshape(0,dimen)
actions = np.empty(0).reshape(0,1)
rewards = np.empty(0).reshape(0,1)
discounted_rewards = np.empty(0).reshape(0,1)

# Setting up our environment
observation = env.reset() # 한개의 state를 발생시킨다.

num_episode = 0

losses = []

while num_episode < num_episodes:
    # Append the observations to our batch
    state = np.reshape(observation, [1, dimen]) # 하나의 state를 (-1, 1, 4) array로 만든다.
    
    predict = model_predict.predict([state])[0]  # 한개의 state를 inout해서 신경망을 통해 action을 만들어낸다.
    action = np.random.choice(range(num_actions),p=predict) # 대안 중에서 하나를 선택한다. p=[0.4, 0.6] 대안이 선택될 확율이다. action은 0 또는 1 이된다.
    
    # Append the observations and outputs for learning
    states = np.vstack([states, state]) # 미리 만들어 놓은 placeholder인 states에 while문 밖에서 만들어낸 state를 append 시키다.
    actions = np.vstack([actions, action]) # 동일하게 state가 입력이 되어서 신경망이 산출한 행동을 무작위로 선택한 행동을 actions에 append한다.
    
    # Determine the oucome of our action
    observation, reward, done, _ = env.step(action)
    reward_sum += reward
    rewards = np.vstack([rewards, reward])
    
    if done:
        # Determine standardized rewards
        discounted_rewards_episode = discount_rewards(rewards, gamma)       
        discounted_rewards = np.vstack([discounted_rewards, discounted_rewards_episode])
        
        rewards = np.empty(0).reshape(0,1)

        if (num_episode + 1) % batch_size == 0:
            discounted_rewards -= discounted_rewards.mean()
            discounted_rewards /= discounted_rewards.std()
            discounted_rewards = discounted_rewards.squeeze()
            actions = actions.squeeze().astype(int)  # squeeze 함수는 배열에서 차원이 1인 것을 찾아서 없애버린다. 예) (2,2,1) --> (2,2)
           
            actions_train = np.zeros([len(actions), num_actions])
            actions_train[np.arange(len(actions)), actions] = 1
            
            loss = model_train.train_on_batch([states, discounted_rewards], actions_train)  #####
            losses.append(loss)

            # Clear out game variables
            states = np.empty(0).reshape(0,dimen)
            actions = np.empty(0).reshape(0,1)
            discounted_rewards = np.empty(0).reshape(0,1)


        # Print periodically
        if (num_episode + 1) % print_every == 0:
            # Print status
            score = score_model(model_predict,10)
            print("Average reward for training episode {}: {:0.2f} Test Score: {:0.2f} Loss: {:0.6f} ".format(
                (num_episode + 1), reward_sum/print_every, 
                score,
                np.mean(losses[-print_every:])))
            
            if score >= goal:
                print("Solved in {} episodes!".format(num_episode))
                break
            reward_sum = 0
                
        num_episode += 1
        observation = env.reset()

Average reward for training episode 100: 27.47 Test Score: 110.70 Loss: -0.003320 
Solved in 99 episodes!


In [ ]:

env.close()

In [ ]:
import numpy as np

In [ ]:
y= np.empty(0).reshape(0,1)

In [ ]:
y

array([], shape=(0, 1), dtype=float64)

In [ ]:
env.action_space.n

2

In [ ]:
observation = env.reset()

In [ ]:
observation

array([-0.04977838,  0.04408428, -0.01042855,  0.003663  ])

In [ ]:
state = np.reshape(observation, [1, dimen])

In [ ]:
dimen

4

In [ ]:
state

array([[-0.04977838,  0.04408428, -0.01042855,  0.003663  ]])

In [ ]:
 predict = model_predict.predict([state])[0]

In [ ]:
predict

array([0.4965743, 0.5034257], dtype=float32)

In [ ]:
#num_actions = env.action_space.n
action = np.random.choice(range(num_actions),p=predict)
print(action)

1


In [ ]:
action

0

In [ ]:
np.random.choice([1,2,3], size=1, replace=True, p=(0.1, 0.5, 0.4))

array([3])

In [ ]:
states = np.empty(0).reshape(0,dimen)
states = np.vstack([states, state])

In [ ]:
states

array([[-0.04977838,  0.04408428, -0.01042855,  0.003663  ]])